In [1]:
import pandas as pd
import numpy as np
import re
from textblob import TextBlob
from datetime import datetime

# Configuración para mostrar todas las columnas
pd.set_option('display.max_columns', None)

print("Librerías importadas correctamente.")

ModuleNotFoundError: No module named 'textblob'

In [ ]:
# 1. Cargar los datasets originales
# Asumimos que los archivos tienen extensión .csv
df_luisa = pd.read_csv('data//Comentarios_Extraidos - Comments_Luisa.csv')
df_noboa = pd.read_csv('data//Comentarios_Extraidos - Comments_Noboa.csv')

# Agregar una columna temporal para identificar la fuente (opcional, pero útil para depurar)
df_luisa['candidate_target'] = 'Luisa Gonzalez'
df_noboa['candidate_target'] = 'Daniel Noboa'

# 2. Unir los dataframes (Concatenación)
df = pd.concat([df_luisa, df_noboa], ignore_index=True)

# 3. Eliminar duplicados exactos (basado en tweetId)
initial_len = len(df)
df = df.drop_duplicates(subset=['tweetId'], keep='first')
print(f"Filas iniciales: {initial_len}, Filas tras eliminar duplicados: {len(df)}")

# Reiniciar el índice
df.reset_index(drop=True, inplace=True)
df.head(3)

In [ ]:
def parse_twitter_date(date_str):
    if pd.isna(date_str):
        return pd.NaT
    # Limpiamos espacios extra
    date_str = str(date_str).strip()
    # Formato: "February 8, 2025 at 01:08 PM"
    # %B = Mes completo, %d = día, %Y = Año, %I = Hora 12h, %M = Minutos, %p = AM/PM
    try:
        return datetime.strptime(date_str, "%B %d, %Y at %I:%M %p")
    except ValueError:
        # En caso de que haya algún formato extraño, retornamos NaT
        return pd.NaT

# Aplicamos la conversión a 'createdAt' y 'authorJoinDate'
print("Procesando fechas... (esto puede tardar unos segundos)")
df['createdAt_dt'] = df['createdAt'].apply(parse_twitter_date)
df['authorJoinDate_dt'] = df['authorJoinDate'].apply(parse_twitter_date)

# Creamos la columna 'Date' simple (solo fecha, sin hora) solicitada
df['Date'] = df['createdAt_dt'].dt.date

# Verificar que no haya NaT (Not a Time)
print(f"Fechas fallidas: {df['createdAt_dt'].isna().sum()}")

In [ ]:
def clean_text(text):
    """Limpieza básica para conteo de palabras"""
    if not isinstance(text, str):
        return ""
    # Quitar URLs
    text = re.sub(r'http\S+', '', text)
    # Quitar caracteres especiales pero dejar puntuación básica
    return text.strip()

def get_sentiment(text):
    """Calcula polaridad con TextBlob (-1 a 1)"""
    if not isinstance(text, str):
        return 0.0
    return TextBlob(text).sentiment.polarity

# --- 1. Variables Temporales ---
# Edad de la cuenta en días al momento del tweet
df['account_age_days'] = (df['createdAt_dt'] - df['authorJoinDate_dt']).dt.days
# Si sale negativo por error de zona horaria o creación el mismo día, poner 0
df['account_age_days'] = df['account_age_days'].apply(lambda x: x if x >= 0 else 0)

# --- 2. Variables de Texto y Estructura ---
# Conteos básicos
df['content_length'] = df['content'].fillna("").apply(len)
df['mentions_count'] = df['content'].fillna("").apply(lambda x: x.count('@'))
df['hashtags_count'] = df['content'].fillna("").apply(lambda x: x.count('#'))

# Ratios y Estadísticas de Texto
def text_features(row):
    text = str(row['content'])
    if not text:
        return pd.Series([0.0, 0.0, 0.0, 0.0])
    
    # Uppercase Ratio (Gritos)
    uppercase_chars = sum(1 for c in text if c.isupper())
    uppercase_ratio = uppercase_chars / len(text) if len(text) > 0 else 0
    
    # Palabras
    words = clean_text(text).split()
    num_words = len(words)
    
    if num_words == 0:
        return pd.Series([uppercase_ratio, 0.0, 0.0, 0.0])
    
    # Unique Word Ratio (Riqueza léxica)
    unique_words = len(set(words))
    unique_word_ratio = unique_words / num_words
    
    # Avg Word Length
    avg_word_length = sum(len(w) for w in words) / num_words
    
    # Mention to word ratio
    mention_ratio = row['mentions_count'] / num_words
    
    return pd.Series([uppercase_ratio, unique_word_ratio, avg_word_length, mention_ratio])

# Aplicar la función de ratios
print("Calculando estadísticas de texto...")
df[['uppercase_ratio', 'unique_word_ratio', 'avg_word_length', 'mention_to_word_ratio']] = df.apply(text_features, axis=1)

# --- 3. Variables de Perfil ---
# has_profile_picture: Asumimos que si la URL contiene "default", es falsa.
# Ajusta esto si tus datos tienen otra lógica (ej. si la URL está vacía)
df['has_profile_picture'] = df['authorProfilePic'].fillna("default").apply(
    lambda x: False if "default" in str(x).lower() else True
)

# --- 4. Sentimiento (Polaridad) ---
# Nota: Esto es preliminar antes de los Intents avanzados
print("Calculando sentimiento (TextBlob)...")
df['sentiment_polarity'] = df['content'].apply(get_sentiment)

In [ ]:
# Crear un diccionario de ID -> Fecha de Creación para búsqueda rápida
tweet_time_map = pd.Series(df.createdAt_dt.values, index=df.tweetId).to_dict()

def calculate_response_time(row):
    """
    Calcula minutos entre el tweet y el tweet al que responde.
    Si no encuentra el padre en el dataset, retorna NaN (o -1).
    """
    if pd.isna(row['inReplyToId']):
        return -1 # No es respuesta o no hay ID
    
    parent_id = row['inReplyToId']
    
    # Buscamos si tenemos la fecha del padre en nuestro dataset descargado
    if parent_id in tweet_time_map:
        parent_time = tweet_time_map[parent_id]
        current_time = row['createdAt_dt']
        
        # Diferencia en minutos
        diff = (current_time - parent_time).total_seconds() / 60.0
        return diff if diff >= 0 else 0
    else:
        # El tweet padre no está en nuestro dataset (ej. tweet del candidato muy antiguo)
        # Aquí decidimos qué hacer: Dejar NaN o poner 0.
        # Para la GNN a veces se imputa con la media, aquí dejaremos NaN para tratarlo luego.
        return np.nan

print("Calculando tiempos de respuesta...")
df['time_response'] = df.apply(calculate_response_time, axis=1)

# Imputar valores nulos en time_response si es necesario (ej. con la media)
# Ojo: Si es NaN significa que no tenemos el padre.
# Para evitar errores en CSV, llenaremos con -1 o 0 según prefieras.
df['time_response'] = df['time_response'].fillna(0)

In [ ]:
# Lista de columnas deseadas en orden
desired_columns = [
    'tweetId', 'tweetUrl', 'content', 'isReply', 'replyTo', 'createdAt',
    'authorId', 'authorName', 'authorUsername', 'authorVerified', 
    'authorFollowers', 'authorProfilePic', 'authorJoinDate', 'source', 
    'hashtags', 'mentions', 'conversationId', 'inReplyToId',
    # Features generadas
    'Date', 'time_response', 'account_age_days', 'mentions_count', 
    'hashtags_count', 'content_length', 'has_profile_picture', 
    'sentiment_polarity', 'uppercase_ratio', 'unique_word_ratio', 
    'avg_word_length', 'mention_to_word_ratio'
]

# Filtramos (asegurando que existan todas)
df_final = df[desired_columns].copy()

# Verificación final
print(f"Shape final: {df_final.shape}")
df_final.head()

In [ ]:
output_filename = 'tweets.csv'
df_final.to_csv(output_filename, index=False, encoding='utf-8')

print(f"✅ Archivo generado exitosamente: {output_filename}")